## Recency, Frequency, and Money leading into K-Means clustering

Chet Huang, 27 Feb 2020

### Purpose
In this notebook we explore sales data from an online retailer.  This data has been posted to UC Irvine's machine learning data repository.  We use this data to identify customer groupings by recency, frequency, and monetary value.  This grouping or segmentation of customers can help the sales and marketing teams plan targeted campaigns to cultivate customer relationships.  Many such campaigns have goals related to improving a company's bottom line.

In [1]:
# Import necessary libraries
import pandas as pd
import datetime as dt
import numpy as np
from collections import OrderedDict
from sklearn.cluster import KMeans

In [2]:
# Read in data file and look at attribute names and data types
file = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
store = pd.read_excel(file)
store.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [3]:
print("Number of rows, columns: ",store.shape)

Number of rows, columns:  (541909, 8)


Let's check the date windows from our dataset.

In [4]:
ts = store.InvoiceDate
EarliestDate = min(store.InvoiceDate)
LatestDate = max(store.InvoiceDate)
print("Earliest date=%s, Latest date=%s", EarliestDate.strftime("%Y-%m-%d"), LatestDate.strftime("%Y-%m-%d"))

Earliest date=%s, Latest date=%s 2010-12-01 2011-12-09


We have just over a year's worth of sales data.  We could set boundaries on the dates of our analysis, but let's go ahead and use all transactions in the dataset.  We'll copy these transactions into a new dataframe and also add a new calculated column for the total Sales which is UnitPrice * Quantity.

In [5]:
#datemask = (store.OrderDate>=EarliestDate) & (store.OrderDate<CalculationDate)
transactionsRfm = store.copy()
transactionsRfm['Sales'] = transactionsRfm.Quantity * transactionsRfm.UnitPrice
transactionsRfm.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


To get recency, we need the last transaction for every customer

In [6]:
lastTransaction = transactionsRfm.groupby('CustomerID').InvoiceDate.max()
lastTransaction.head()

CustomerID
12346.0   2011-01-18 10:17:00
12347.0   2011-12-07 15:52:00
12348.0   2011-09-25 13:13:00
12349.0   2011-11-21 09:51:00
12350.0   2011-02-02 16:01:00
Name: InvoiceDate, dtype: datetime64[ns]

From recency date we calculate day elapsed then convert results to dataframe

In [7]:
rfm = (LatestDate - lastTransaction).dt.days
rfm = rfm.to_frame('Recency')
rfm.head()

,Recency
CustomerID,
12346.0,325
12347.0,1
12348.0,74
12349.0,18
12350.0,309


Frequency requires us to count the number of tranactions by customer

In [8]:
rfm.loc[:,'Frequency'] = transactionsRfm.groupby('CustomerID').InvoiceNo.count()
rfm.head()

,Recency,Frequency
CustomerID,,
12346.0,325,2
12347.0,1,182
12348.0,74,31
12349.0,18,73
12350.0,309,17


Money is a sum of the Sales attribute in the transaction

In [9]:
rfm.loc[:,'Money'] = transactionsRfm.groupby('CustomerID').Sales.sum()
rfm.head()

,Recency,Frequency,Money
CustomerID,,,
12346.0,325,2,0.00
12347.0,1,182,4310.00
12348.0,74,31,1797.24
12349.0,18,73,1757.55
12350.0,309,17,334.40


At this point we have a table of customers, their recency (days since last transaction), frequency of their transactions in the last year, and how much in sales they generated.  This table contains actionable information for our sales and marketing teams.  For example, high frequency & high money customers with high recency (many days since last transaction) may mean that the customer has churned.  Marketing may want to run campaigns to see if we can win these customers back.  Conversely, customers with low frequency, low money, but recent transactions may warrant futher cultivation for their continued patronage.

Let's take our RFM analysis a step further and segment this RFM list into groups to help our sales and marketing teams target relevant campaigns.  Let's use K-Means clustering to segment these customers.  Since we have three attributes, let's take a simple approach and divide each attribute into low or high.  This will effectively split the 3D space into 8 clusters  We will seed the K-Means clustering with centroids located at low and high along each axis.

To properly assign centroids, we also need to define what values constitute low or high.  Since each attribute in R, F, M are not similar in scale, we will use min-max normalization so that each attribute will only vary between 0 and 1.  Then for the centroids we can use 0.25 for low, and 0.75 for high.

In [10]:
def NormalizeColumn(data, columns):
    """
    This function applies min-max normalization.  Inputs are dataframe called
    data and array called columns
    """
    for each in columns:
        data.loc[:,each] = (data.loc[:,each] - np.min(data.loc[:,each])) \
            / (np.max(data.loc[:,each]) - np.min(data.loc[:,each]))
    return data

rfm_norm = NormalizeColumn(rfm, ['Recency','Frequency','Money'])
rfm_norm.head()

,Recency,Frequency,Money
CustomerID,,,
12346.0,0.871314,0.000125,0.015109
12347.0,0.002681,0.022676,0.030297
12348.0,0.198391,0.003758,0.021442
12349.0,0.048257,0.009020,0.021303
12350.0,0.828418,0.002005,0.016288


Now we're ready to use K-Means to find 8 clusters with seed centroids

In [11]:
k_centers = np.array([[0.25,0.25,0.25],[0.25,0.75,0.25],[0.25,0.25,0.75],[0.25,0.75,0.75],
                    [0.75,0.25,0.25],[0.75,0.25,0.75],[0.75,0.75,0.75],[0.75,0.75,0.25]], np.float64)
k_means = KMeans(n_clusters=8,init=k_centers,n_init=1).fit(rfm_norm)

# Let's print the clustered centers to see how far they've drifted from the seed
centers = k_means.cluster_centers_
print(centers)

[[0.29766908 0.00562291 0.01823263]
 [0.00268097 0.74079178 0.24337532]
 [0.02144772 0.12683538 0.69206358]
 [0.90729203 0.00279722 0.01624688]
 [0.48913683 0.00458046 0.01721598]
 [0.15217721 0.00754943 0.01886927]
 [0.69463429 0.0028102  0.01676095]
 [0.03733226 0.01777505 0.0247879 ]]


We see that the cluster centers have drifted quite a bit from the initial centers.  This suggests that the customers were not very evenly distributed across the high-low RFM space.  Such is the nature of our sales data.  Nevertheless, let's get the K-Means cluster labels.

In [12]:
k_means_predicted = k_means.predict(rfm_norm)
k_means_predicted

array([3, 7, 5, ..., 7, 7, 5])

We can now merge these labels back to our original rfm dataframe.

In [13]:
rfm['ClusterID'] = k_means_predicted
rfm

,Recency,Frequency,Money,ClusterID
CustomerID,,,,
12346.0,0.871314,0.000125,0.015109,3
12347.0,0.002681,0.022676,0.030297,7
12348.0,0.198391,0.003758,0.021442,5
12349.0,0.048257,0.009020,0.021303,7
12350.0,0.828418,0.002005,0.016288,3
12352.0,0.093834,0.011776,0.020555,7
12353.0,0.544236,0.000376,0.015423,4
12354.0,0.619303,0.007141,0.018913,6
12355.0,0.571046,0.001503,0.016728,4


## Summary

In this notebook we have read in sales data from an online retailer and performed RFM analysis on customer transactions for the last year.  From the RFM analysis we further segmented the customers using K-Means clustering.  There were 8 clusters, made up of upper and lower tiers along the Recency, Frequency, and Money axes.  This customer segmentation allows our sales and marketing teams to target campaigns to respective customer groups.  For example, high frequency customers who spent high amounts but have low recency may be churning.  Marketing should act upon this information and seek out ways to cultivate these high value customers.